# SIG13 GEP Scoring

Here, I will calculate GEP scores for the sPCA components using a weighted mean of scaled log1p_norm component genes.

In [ ]:
%load_ext autoreload
%autoreload 2

import scanpy as sc
import pandas as pd
import decoupler as dc
import matplotlib as mpl
import matplotlib.pyplot as plt

import sys
sys.path.insert(0, '/data1/rudenska/EYW/git_projects/SIG13/functions')
import scanpy_custom as scc

In [ ]:
rna = sc.read_h5ad("/data1/rudenska/EYW/SIG13/scanpy_outs/SIG13_doublets_DSB7.h5ad")

In [2]:
spca_components = pd.read_csv("/data1/rudenska/EYW/git_projects/SIG13/analysis_outs/spca/zscore_degs_allLigands_0.1_alpha1.0_sPCA_loadings.csv")
net = spca_components.rename(columns={'gene':'target',
                                      'spca_component':'source',
                                      'loading':'weight'})

# define genes used in scoring
comp_genes = net['target'].unique().tolist()

In [ ]:
# subset and scale 
rna_sub = rna[:,comp_genes].copy()
rna_sub.X = rna_sub.layers['log1p_norm']
sc.pp.scale(rna_sub)

/home/wange7/miniforge3/envs/scanpy_standard2/lib/python3.13/functools.py:934: UserWarning: zero-centering a sparse array/matrix densifies it.
  return dispatch(args[0].__class__)(*args, **kw)


In [13]:
# calculate pseudobulk 
rna_pb = sc.get.aggregate(rna_sub, by=['ligand_call_DSB7','replicate'], func='mean')

In [14]:
dc.mt.waggr(rna_pb, net, tmin=5, verbose=True, layer='mean', times=0)

2026-01-09 13:04:04 | [INFO] waggr - Running waggr
2026-01-09 13:04:04 | [INFO] Extracted omics mat with 1320 rows (observations) and 6610 columns (features)
2026-01-09 13:04:04 | [INFO] Network adjacency matrix has 6610 unique features and 78 unique sources
2026-01-09 13:04:04 | [INFO] waggr - using function _wmean
2026-01-09 13:04:04 | [INFO] waggr - calculating scores for 78 sources across 1320 observations
2026-01-09 13:04:05 | [INFO] waggr - adjusting p-values by FDR
2026-01-09 13:04:05 | [INFO] waggr - done


In [15]:
# Extract score_waggr and padj_waggr from rna_pb.obsm
score_df = pd.DataFrame(rna_pb.obsm['score_waggr'], 
                        index=rna_pb.obs_names, 
                        columns=rna_pb.obsm['score_waggr'].columns)

# Calculate differential scores by subtracting linker_linker scores from all conditions
score_df = score_df.rename_axis('interaction')

In [16]:
score_df.to_csv("/data1/rudenska/EYW/git_projects/SIG13/analysis_outs/spca/degs_zscore_allLigands/zscore_degs_allLigands_0.1_alpha1.0_waggr_score.csv")